### This is the second version of the crypto_scrapper. In this script I have added some extra lines of code that stores the crypto data inside a database

#### Importing the relevant libraries

In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3
from datetime import date,datetime

#### The data is scrapped from 'https://www.coingecko.com/' and it is then stored inside a Pandas DataFrame and then written into a csv file as well as database called crypto_data_test

In [10]:
def crypto_scrapper():
    current_date=datetime.now()
    html_script=requests.get('https://www.coingecko.com/').text
    soup=BeautifulSoup(html_script,'lxml')
    lst=soup.find_all('a',class_='d-lg-none font-bold tw-w-12')
    rates=soup.find_all('td', class_='td-price price text-right pl-0')
    caps=soup.find_all('td', class_='td-market_cap cap col-market cap-price text-right')
    names=[]
    prices=[]
    market_cap=[]
    dates=[]
    for name in lst:
        names.append(name.text.replace('\n',''))
        dates.append(current_date)
    for rate in rates:
        prices.append(rate.text.replace('\n','').replace('$','').strip().replace(',',''))
    for cap in caps:
        market_cap.append(cap.text.replace('\n','').replace('$','').strip().replace(',',''))
    #since the market_cap and price variable is a string, we need to convert it into float
    market_cap_float=list(map(float,market_cap))
    price_float=list(map(float,prices))
    crypto_frame=pd.DataFrame([dates,names,price_float,market_cap_float])
    crypto_frame_new=crypto_frame.transpose()
    crypto_frame_new.columns=['date','name','price_in_usd','market_cap']
    crypto_frame_new.to_csv(f"{date.today()}.csv", index=False)
    # storing data into a database
    conn=sqlite3.connect('crypto_data_test.db')
    crypto_frame_new.to_sql(name="crypto_data_table",con=conn, if_exists="append", index=False)
    conn.commit()
    conn.close()
    

    

    
crypto_scrapper()






